In [ ]:
import os
import sys
import yaml
import json
import numpy as np
import pandas as pd
import torch
import torch.utils.data
import torchvision

%matplotlib inline
import matplotlib.pyplot as plt
import cv2
from PIL import Image

plt.rcParams["figure.figsize"] = (20,10)

torch.cuda.is_available()

True

In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-dvbxln54
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-dvbxln54
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263919 sha256=bf619ef5d4e5c74291375aff7d8ecdc18c53d58085268a3d66ee5167e0f9cfea
  Stored in directory: /tmp/pip-ephem-wheel-cache-i420guho/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


In [ ]:
!git clone https://github.com/aapanaetov/Traffic_Light_Detection.git

fatal: destination path 'Traffic_Light_Detection' already exists and is not an empty directory.


In [ ]:
os.listdir('drive/MyDrive/Detection/weights')

['res50FPN_final.pth', 'mobilenet_v3_final.pth']

In [ ]:
import sys
sys.path.append('Traffic_Light_Detection/')
import utils
from model import get_model
from dataset import BoschDataset, LisaDataset
from dataset import get_transform, label_to_class
from engine import train_one_epoch, evaluate

In [ ]:
# Choose the model to test

device = torch.device('cuda')

model = get_model('mobilenet_v3_FPN').to(device)
model.load_state_dict(torch.load('drive/MyDrive/Detection/weights/mobilenet_v3_final.pth'))

# model = get_model('res50_FPN').to(device)
# model.load_state_dict(torch.load('drive/MyDrive/Detection/weights/res50FPN_final.pth'))

<All keys matched successfully>

In [ ]:
# Change paths to save json results and video visualization somewhere else if you want

def make_prediction(video_path, model, save_video=False):
    video_name = video_path[video_path.rfind('/')+1:-4]
    save_result_path = 'drive/MyDrive/{}.json'.format(video_name)
    if save_video:
        save_video_path = 'drive/MyDrive/{}.mp4'.format(video_name)

    device = torch.device('cuda')
    model.eval()
    
    frame_array = []
    result_video = {}
    count = 0
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    while cap.isOpened():
        success, image = cap.read()
        if success:
            height, width, _ = image.shape
            size = (width, height)
            result_frame = {}

            with torch.no_grad():
                if image.max() < 2:
                    print('WRONG NORMALIZATION')
                    break
                prediction = model([torch.from_numpy((image / 255.).transpose((2,0,1))).type(torch.FloatTensor).to(device)])[0]
            
            n_pred = len(prediction['scores'].cpu().detach().numpy())
            traffic_light = 0
            for i in range(n_pred):
                if prediction['scores'][i].cpu().detach().numpy() > 0.5:
                    bbox = prediction['boxes'][i].cpu().detach().numpy().astype(np.uint16)
                    label = label_to_class(int(prediction['labels'][i].cpu().detach().numpy()))
                    
                    traffic_light_dict = {}
                    traffic_light_dict['coords'] = bbox.tolist()
                    traffic_light_dict['state'] = label
                    traffic_light_dict['affect'] = True
                    result_frame['traffic_light_{}'.format(traffic_light)] = traffic_light_dict
                    traffic_light += 1
                    
                    if save_video:
                        image = cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (36,255,12), 2)
                        image = cv2.putText(image, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                        
            result_video[str(count)] = result_frame
            count += 1
            
            if save_video:
                frame_array.append(image)
        else:
            break

    with open(save_result_path, 'w') as f:
        json.dump(result_video, f)
                
    if save_video:
        out = cv2.VideoWriter(save_video_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
        for i in range(len(frame_array)):
            out.write(frame_array[i])
        out.release()

In [ ]:
# Use the following two lines to predict results for all videous in test_videos folder

# for video_path in os.listdir('drive/MyDrive/Detection/test_videos'):
#     make_prediction(video_path='drive/MyDrive/Detection/test_videos/'+video_path, model=model, save_video=True)

make_prediction(video_path='drive/MyDrive/Detection/test_videos/video_0.mp4', model=model, save_video=True)

In [ ]:
# The following two cells just to test inference time 
# You can download any other image instead so u don't have to download the whole Bosch Dataset

# ds = BoschDataset(get_transform(train=True))
# sample = ds.samples[0]
# image = plt.imread(sample['path'])
image = np.zeros((800, 1200, 3))

In [ ]:
%%time

device = torch.device('cuda')
model.eval()

for i in range(100):
    with torch.no_grad():
        prediction = model([torch.from_numpy(image.transpose((2,0,1))).type(torch.FloatTensor).to(device)])[0]

# So we have almost 37 FPS

CPU times: user 2.34 s, sys: 403 ms, total: 2.74 s
Wall time: 2.74 s


In [ ]:
# The cells below show how to set up training
# You should download datasets to be ablo to run this code

Bosch_train = BoschDataset(get_transform(train=True))
Lisa_train = LisaDataset(get_transform(train=True))
Bosch_test = BoschDataset(get_transform(train=False))
Lisa_test = LisaDataset(get_transform(train=False))

dataset = torch.utils.data.ConcatDataset([Bosch_train, Lisa_train])
dataset_test = torch.utils.data.ConcatDataset([Bosch_test, Lisa_test])

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=8, shuffle=True, collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda')

model = get_model('mobilenet_v3_FPN').to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
num_epochs = 4

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

  torch.save(model.state_dict(), 'weights/mobilenet_v3_4e_Bosch_Lisa.pth')